# Path Setting Tool Specification Document

## Overview

We want to create a Mercury implementation of the **Path Setting Tool** to replace the excel implementation so that we can run this tool in sequence in our calibrations.    

The **Path Setting Tool** that we require can be thought of as a wrapper around **4** runs of a basic path setting function.  

We can show the proposed TMT model flow for the Mercury implentation of the **Path Setting Tool**. Note that this uses exactly the same Models as the excel implementation, with the single addition of a *Settings.RealWorldPathSettingTool* model, to expose the settings as an input TMT model.  

![](images/TMT_PathSetting.svg)

From the outputs, we can see that we will create **four** separate paths in this tool. All of these can be produced from a common *general case* of a path setting function. The job of this tool will be to "sort" the data from the input models, create inputs to the 4 instances of the general case function, and return the output in each case to the appropriate output Model.  

## Path Setting - general case as a function

The general form of the path setting function is:

$$ r_t = [S_t(1-\alpha)^t+M_t(1-(1-\alpha)^t)](1-k_t)+k_tL_{unc} $$

Where:

$ k_t $ is the exponential (logistic) weight, according to the function

$$ k_t = \frac{1}{1+\exp^{-Bt+A}}-\frac{1}{1+\exp^A} $$

$ \alpha $ is the global "Fully Justified" speed of mean reversion.

$ S_t $ is the short term path appropriate to the target path  

$ M_t $ is the medium, term path, according to the function:

$$ M_t = X(1-\alpha)^t + Y(1-(1-\alpha)^t) $$  

> In practice, this is the same global value for  $ \alpha $

$ L_{unc} $ is the long term target for the path - in practice the *unconditional* target for the path.

### Visual Example

This can be illustrated as below

![](data\PathsPlot.png)

### Implementation example in F#  ###

#### Helper function for evaluating paths

To evaluate a function over a set of given terms, I have created a helper function:

```fsharp

  // Helpers - to evaluate a function over a given set of points in time
  let evalPointsAtTerms terms pointFun = terms |> Array.map pointFun
  let evalPointsWithTerms terms pointFun = terms |> Array.map (fun t -> (t, pointFun t))
  
```

so evaluating an array $ t[] $ using the function $ f(t[]) = f_t $

#### Short Term Paths

The short term path $ S_t $ will be a direct input into the function. This will take the form of an input curve of the same type as the desired output.  

We will cover the exact requirement for each of the four instances we will run later.   

#### Logistic weights 

The logistic weighting function $ k_t = \frac{1}{1+\exp^{-Bt+A}}-\frac{1}{1+\exp^A} $ can be implemented as:

``` fsharp

  // logistic weight function
  type LogisticFunctionParams = {A: float; B: float}

  let logisticWeight (logistic: LogisticFunctionParams) term = 
    1.0 / (1.0 + exp(-logistic.B * term + logistic.A)) - 1.0 / (1.0 + exp(logistic.A))

  let logisticWeights (logistic: LogisticFunctionParams) terms = 
    evalPointsWithTerms terms (logisticWeight logistic) 

```

where $ k = logisticWeights =  f(logisticFunctionParams, terms) $

The $ terms $ will be an array we derive from the length of the short term path $ S $

#### Medium Term Path

The Medium term path $ M $ is derived as an autoregressive function of order 1, or $ AR(1) $.  

The function for each point in the path, $ M_t = X(1-\alpha)^t + Y(1-(1-\alpha)^t) $  can be implemented as:

``` fsharp

  // Autoregressive order 1 (AR(1)) function - we use this to calculate the "Medium" path in each case

  type AR1FunctionParams = {initialVal:float; targetVal:float; meanRevRate:float}  

  let rateAR1 initialVal targetVal meanRevRate term = 
    initialVal * (1.0 - meanRevRate) ** term + targetVal * (1.0-(1.0-meanRevRate)**term)

  let rateAR1Function (inputs: AR1FunctionParams) = 
    rateAR1 inputs.initialVal inputs.targetVal inputs.meanRevRate

  let rateAR1Path (inputs: AR1FunctionParams) terms = 
   let baseAR1Function = rateAR1Function inputs
   evalPointsWithTerms terms baseAR1Function 


```

The intended use is $ M = rateAR1PathFunction = f(AR1pathParams) $

We will need to define the *initialVal* and *targetVal* for each use case of the path setting function.  


So rather than defining the value of $ M $ as a path explicitly in each case, we can define this in terms of the $ X $ and $ Y $ values -  the *initialVal* and *targetVal* for the case.

In practice:

The *initialVal*, or $ X $ will be the $ r_0$ value of the short term value of the rate we are trying to target - so $ X = S_0 $  

The *targetVal*, or $ Y $ will be the **ECCA** estimate of the medium-term target, as processed from **ECCA** medum-term forecasts.

#### Long Term (Unconditional) rate

$ L_{unc} $ is the long term target for the path - in practice the *unconditional* target for the path. This will always be a single point-in-time value, rather than a path.  

#### Output Target Path

The path setting function itself, $ r_t = [S_t(1-\alpha)^t+M_t(1-(1-\alpha)^t)](1-k_t)+k_tL_{unc} $, can be implemented as follows:

``` fsharp

  // Whole path Targets  
   
  let targetPathPoint weight meanRevRate shortRate mediumRate longRate term =
    ((shortRate * (1.0 - meanRevRate) ** term + mediumRate *  (1.0-(1.0-meanRevRate)**term)) * (1.0 - weight)) + weight * longRate

  type TargetPathInputs = {weightFunParams:LogisticFunctionParams; meanRevRate: float; shortPath:(float * float)[]; mediumPath:(float * float)[]; longRate:float}  

  let weightedTargetPath weightPath meanRevRate shortPath mediumPath longRate = 
    let terms = shortPath |> Array.map (fst)
    let shortRates = shortPath |> Map.ofArray
    let mediumRates = mediumPath |> Map.ofArray
    let weights = weightPath |> Map.ofArray  
    terms |> Array.map (fun t -> (t, targetPathPoint weights.[t] meanRevRate shortRates.[t] mediumRates.[t] longRate t))  

  let targetPath weightFunParams meanRevRate shortPath mediumPath longRate = 
    let terms = shortPath |> Array.map (fst)
    let weights = logisticWeights weightFunParams terms   
    weightedTargetPath weights meanRevRate shortPath mediumPath longRate

  let targetPathFunction (inputs:TargetPathInputs) =
    targetPath inputs.weightFunParams inputs.meanRevRate inputs.shortPath inputs.mediumPath inputs.longRate
    
```

Putting this all together, we can derive a function that works as a "Tool"  


i.e a function of the form $ outputs = f(inputs) $


``` fsharp

  // Alternatively, we can define the medium path in terms of the AR(1) target only
  type TargetPathToolInputs = 
    {  
      weightFunParams:LogisticFunctionParams; 
      meanRevRate: float; 
      shortPath:(float * float)[]; 
      mediumPathTarget:float; 
      longRate:float
    }

  // Return the final output as a record type. We will include all the evaluated intermediate parameters within this
  type TargetPathToolOutputs = 
    {
      targetPath:(float * float)[]; 
      weights:(float * float)[]; 
      meanRevRate: float;
      shortPath:(float * float)[]; 
      mediumPath:(float * float)[]; 
      longRate:float
    }  

  let targetPathTool (inputs:TargetPathToolInputs) = 
    // Take terms from the short rate path
    let terms = inputs.shortPath |> Array.map (fst)
    // Expand using an AR(1) function for the medium term
    let initialMediumVal = inputs.shortPath.[0] |> snd
    let mediumPath = rateAR1Path {initialVal = initialMediumVal;targetVal = inputs.mediumPathTarget; meanRevRate = inputs.meanRevRate} terms 
    // Evaluate the weights from the logistic function 
    let weights = logisticWeights inputs.weightFunParams terms 
    let targetPath = weightedTargetPath weights inputs.meanRevRate inputs.shortPath mediumPath inputs.longRate
    // Compile the outputs and intermediate parameters into a single output record
    {targetPath=targetPath; weights=weights; meanRevRate= inputs.meanRevRate; shortPath = inputs.shortPath; mediumPath = mediumPath; longRate = inputs.longRate}
  
  

```

Note that as well as the required final *targetPath*, we include all of the intermediate calculations as outputs. If we are to create UIs from *Models* only, we will need to make sure that all required values are included as outputs of the methods that we create.

## Required Output Paths

As shown in the TMT diagram, the tool will output **four** target paths in total.  

We will set target paths for two components of our interest rate modelling, i.e. for **Nominal** and **Real** interest rate models.  
For each model, we will require two targets - one for the **Short Term** and one for the **Long Term**.  

Combining these gives us our four use cases:
+ Nominal Short Term
+ Nominal Long Term 
+ Real Short Term
+ Real Long Term

For each case, we can spell out:

+ What label we have used in our TMT diagram above
+ What type of rate we will require as the **Output Target Path**
+ What we will use as the **Short Rate Path **
    + $ S_t $ in our definitions above
    + We will spell out in detail how these will be calculated from the given inputs later.
+ The **Medium Term** target value
    + This is the value of $ Y $ in the function for $ M_t $
    + This will be a value provided by analysis of ECCA forecast data, so we will use the model parameter name here
+ The **Unconditional Target** value
    + $ L_{unc} $ in our definitions above
    + Some of these are direct inputs from the input models, others will need to be calculated from the inputs
+ The **Mean Reversion Rate** used ($ \alpha $)
    + This will be the same global assumption in each case, for both the $ r_t $ and $ M_t $ functions.

We can sum up the choices for each path in a table below.

In [15]:
import pandas as pd
df = pd.read_json("data/PathSettingCases.json")
df.set_index(["Required"], inplace=True)
df[['Nominal Short Term','Nominal Long Term','Real Short Term','Real Long Term']]

,Nominal Short Term,Nominal Long Term,Real Short Term,Real Long Term
Required,,,,
Label in TMT diagram,NominalPath_1Y,NominalPath_10Y,InflationPath_1Y,RealPath_10Y
Output Target Path,Nominal 1Y Cont. Comp. Rate,Nominal 10Y Ann. Comp. Rate,Expected Cont. Comp. Inflation,Real 10Y Cont. Comp. Rate
Short Rate Path,Nominal Cont. Comp Forward Rates,Forward Rate-Implied 10Y Nominal Rate (Ann. Co...,Initial Break-Even Inflation (Cont. Comp.),Forward Rate-Implied 10Y Real Rate (Cont. Comp.)
Medium Term Target,ECCA.Econ.Nominal.ShortRate.MT.Target,ECCA.Econ.Nominal.LongRate.MT.Target,ECCA.Econ.Inflation.MT.Target,ECCA.Econ.Real.LongRate.MT.Target
Unconditional Target,Input Nominal Unconditonal Short Rate Assumption,Observed 100Y value from a Ed2FBK model path,Input Unc. Inflation Unc. Assumption,Real 10Y Unc. Rate derived from Vasicek Model
Mean Reversion Rate ( α ),Global (nominal) Mean Reversion Speed,Global (nominal) Mean Reversion Speed,Global (nominal) Mean Reversion Speed,Global (nominal) Mean Reversion Speed


In the rest of this spec document, we will go into detail of each of the "required" inputs to each target path, and show how we will derive the values we need from the input models.  

## Mean Reverson rate (α)

Lets take the simplest one first - the Mean Reversion Rate, denoted as $\alpha $ in the formulas above will be a direct input in each of the four cases.  

We will take this value as the **FEA_Nom_MRSpeed** Parameter value from the anchored **Assumptions_MRSpeed** Model.  

We use the same value of $ \alpha $ when calculating both the intermediate **Medium Term** path, and the final **Target Path**.

##### Mocking this up in F# #####

To illustrate extracting Parameters from the input anchored Models of the tool, I have created a module containing mockups of this data.  

For each anchored model, I have created a custom F# *type* with the same name as the anchored model, that contains the parameters that will be required in our **Path Setting Tool** method, with the same names as they will be found in the actual model. 
The model may contain more parameters than this in practice, but as they are not required here, they can be ignored.  

``` fsharp

  // Declaration of type matching anchored model
  type Assumptions_MRSpeed = {``FEA_Nom_MRSpeed``:float}
  
  // Instance of the type containing example data
  let assumptions_MRSpeed_ModelData = {``FEA_Nom_MRSpeed`` = 0.0920681319618401}
  
  // Use of model parameter in the path setting tool mockup
  // mean rev rate is a global setting from the assumptions
  let globalMeanRevRate = assumptions_MRSpeed_ModelData.``FEA_Nom_MRSpeed``
  
```

For each of the required inputs, I will use this approach to show how we will use the input Model data in the functionality of the path setting tool.  


## Short Rate Path

We require an input short rate path for each Target Path. As the Nominal and Real paths are linked, and the long term paths are informed by the short term path, we will need to go through these in order.  

### Nominal Short Term - Short Rate Path

Here we require **Nominal Cont. Comp Forward Rates**

We will take the *Curve.YC.ZCB* parameter from the *nominalShortPathSource* Anchored Model, which is a set of zero coupon bond prices. We will then need to convert these to forward rates using a continuously compounded basis.  

This *should* exist within the yield curves class in the Ghost project, taking care that the compounding is correct. If not, then see the example project for the definition of this.  

Using the YieldCurves "class" included in the example project, we can create the **Nominal Cont. Comp Forward Rates** like so:

``` fsharp

let initial1YNominalFwdRates = 
  nominalShortPathSource_ModelData.``Curve.YC.ZCB``
  |> ZCBCurve
  |> continuouslyCompoundedForwardCurve
  |> unpackFwdCurve

```

### Nominal Long Term - Short Rate Path

Here we require **Forward Rate-Implied 10Y Nominal Rate, annually compounded**.  

I'm pretty sure that this doesn't exist within the yield curves class in the Ghost project, so we will need to refer to the YieldCurves "class" included in the example project for a definition of how this is calculated.  

Given the **Nominal Cont. Comp Forward Rates** calculated above, and denoting these as $ f_t $, we can calculate the **Forward Rate-Implied 10Y Nominal Rate, annually compounded.**, denoted $ f^{[10]}_t $ as follows:

$$ f^{[10]}_t = exp[\frac{\sum\limits_{i=t}^{t+10}f_i}{10}]-1 $$

To put that into words - it is the average of the next 10 continuously compounded annual forward rates, then converted to an annually compounded rate.  

Note that we assume that $ f_t $ is constant if $ t $ is greater than the maximum length of the original forward curve. This is slightly different to the implementation in excel which allows the average to be over whatever curve is left. As this is used as the medium term target, and our curves extend to 120 years, this will make no material difference to the resulting Target Path.  

This can be implemented like so, in the example project.  

``` fsharp

  // from "1-YieldCurves.fs"
  let nYearAnnCompNominalRatePath n forwardRates =
    let forwards = unpackFwdCurve forwardRates
    let terms = forwards |> Array.map (fst)
    let rates = forwards |> Array.map (snd)
    let lastRate = Array.last rates
    let rateBuffer = [|1..n-1|] |> Array.map (fun _ -> lastRate)
    let ratesPlus = Array.append rates rateBuffer
    let nYearCCFwds = ratesPlus |> Array.toSeq |> Seq.windowed 10 |> Seq.map (Array.average >> (fun a -> exp(a)-1.0) ) |> Seq.toArray
    Array.zip terms nYearCCFwds
    
```

And with that function avaliable, we can calculate the **Forward Rate-Implied 10Y Nominal Rate, annually compounded** as a function of the **Nominal Cont. Comp Forward Rates** calculated as the Nominal Short Term Short Rate path.  

``` fsharp

let nominal10YFwdImpliedPath = 
  initial1YNominalFwdRates
  |> ForwardCurve
  |> nYearAnnCompNominalRatePath 10
  
```



### Real Short Term - Short Rate Path

For the real rates, we will express the Short Rate Path in terms of the **Expected Cont. Comp. Inflation Rate**.  

To calculate Inflation, we will need to first derive a path of **Real Cont. Comp Forward Rates**. What we have avaliable to us for real rates is the *Paths.BaseSpotRate* from the *RealShortPathSource* Anchored Model. We will assume that this spot curve is continuously compounded, and convert it to **Real Cont. Comp Forward Rates** (via conversion to ZCBP).  

This conversion may be avaliable in the yield curve class. It is also used when reading parameters in the **Real Hull-White TVTP** tool, as we use the same input. It is also defined in full in the example project.   

The inflation rate $ i_t $ can be defined in terms of the continuously compounded nominal $ f^{[nom]}_t $ and real $ f^{[real]}_t $ forward rates as.

$$ i_t  = f^{[nom]}_t - f^{[real]}_t $$

This is called in the example project like so.

``` fsharp

let real1YForwardRates = 
  realShortPathSource_ModelData.``Paths.BaseSpotRate``
  |> SpotCurve
  |> spotsToZCB Continuous
  |> continuouslyCompoundedForwardCurve
  |> unpackFwdCurve

let breakEvenInflation = 
  Array.zip initial1YNominalFwdRates real1YForwardRates |> Array.map (fun ((t, nom), (_, real)) -> (t, nom - real))
  
```

### Real Long Term - Short Rate Path

Similar to the nominal path, we require a **Forward Rate-Implied 10Y Real Rate**, but this time **continuously compounded**.  

The formula for this is much the same. With  the Given the **Real Cont. Comp Forward Rates** calculated above, and denoting these as $ f_t $, we can calculate the **Forward Rate-Implied 10Y Real Rate**, denoted $ f^{[10]}_t $ as follows:

$$ f^{[10]}_t = \frac{\sum\limits_{i=t}^{t+10}f_i}{10}] $$

To put that into words - it is the average of the next 10 continuously compounded annual forward rates. Unlike in the nominal case, we are already using the compounding convention we want, so no further conversion is required.  

Note that we assume that $ f_t $ is constant if $ t $ is greater than the maximum length of the original forward curve. This is slightly different to the implementation in excel which allows the average to be over whatever curve is left. As this is used as the medium term target, and our curves extend to 120 years, this will make no material difference to the resulting Target Path.  

This is implemented as follows in the example project.  

``` fsharp

  // from "1-YieldCurves.fs"
  let nYearContCompNominalRatePath n forwardRates =
    let forwards = unpackFwdCurve forwardRates
    let terms = forwards |> Array.map (fst)
    let rates = forwards |> Array.map (snd)
    let lastRate = Array.last rates
    let rateBuffer = [|1..n-1|] |> Array.map (fun _ -> lastRate)
    let ratesPlus = Array.append rates rateBuffer
    let nYearCCFwds = ratesPlus |> Array.toSeq |> Seq.windowed 10 |> Seq.map (Array.average) |> Seq.toArray
    Array.zip terms nYearCCFwds
    
```

And with that function avaliable, we can calculate the **Forward Rate-Implied 10Y Real Rate, continuously compounded**. as a function of the **Real Cont. Comp Forward Rates** calculated as the Real Short Term Short Rate path.  

``` fsharp

let real10YFwdImpliedPath =
  real1YForwardRates
  |> ForwardCurve
  |> nYearContCompNominalRatePath 10

```
  

## Medium Term Targets

Recall that:

$ M_t $ is the medium, term path, according to the function:

$$ M_t = X(1-\alpha)^t + Y(1-(1-\alpha)^t) $$  

So for each Target Path, we need to define the value of $ X $, $ Y $ and $ \alpha $

The $ \alpha $ value is set globally. We always use the *FEA_Nom_MRSpeed* parameter from the *Assumptions_MRSpeed* model.

Like so in the example project.

``` fsharp
// mean rev rate is a global setting from the assumptions
let globalMeanRevRate = assumptions_MRSpeed_ModelData.``FEA_Nom_MRSpeed``
```

The $ X $ value is always set to the time zero value of the **Short Term Path**, denoted $ S_0 $. In our "Tool" implementation above we do not need to declare this explicitly, as it will take $ S_0 $ when required from the input **Short Term Path**.  

What remains is to declare the $ Y $ value, which will be our target for the long end of the **Medium Term Target Path**.  All of these will be entered as values from the **MediumPathTargets** anchored model, with the parameter name as in the table above.  

## Unconditional Target

This varies in complexity from case to case, as we will show below.

### Nominal Short Term Path - Unconditional Target

Nice and straighforward to start with - here the unconditional rate is the *FEA_Exp_NomRate* parameter from the *Assumptions_LongRates* anchored model.  

### Nominal Long Term Path - Unconditional Target

We need to take the unconditional path from an existing projection of a Nominal Long Term path. Here we are passing in an existing calibrated Ed2FBK model from an earlier stream - "Best_Views", taking the Long Term Path from this model, and taking the value at 100 years as the Unconditional Target.  

This makes the Unconditional Target the value at year = 100 of the *Paths.LongRate.Target* parameter, from the model passed in through the *NominalLongTargetSource* Anchored Model.  

The implementation of this in the example project is as follows:

``` fsharp

let nominal10YLongRate = 
  nominalLongTargetSource_ModelData.``Paths.LongRate.Target``
  |> Map.ofArray
  |> Map.find 100.0

``` 

### Real Short Term Path - Unconditional Target

Our Real Short Term Path is in terms of an inflation target. Here the unconditional target is the *FEA_Exp_InfRate* parameter from the *Assumptions_LongRates* anchored model.  

### Real Long Term Path - Unconditional Target

This is the tricky one.  

For the **Real Short Term** path, we pass in the result of a *market-consistent* fit to a "real yield curve" using the Vasicek model to the *realShortPathSource* anchored model, and take the *Paths.BaseSpotRate* parameter. For our **Real Long Term Path**, we can take the short term Vasicek fit from this model - so taking the parameters of this model - and use a *real-world* version of this model calibration to find the projected unconditional real long rate.   

The correct place to add this to the project would be in the Vasicek model in Ghost.  

In the example project, I have implemented this in its own **Vasicek** module. The formulas are fiddly, but this is all analyitic functions that can be tested.  

The key takeaways from this implementation are:

+ A custom *type*, **VasicekParameters** to take the parameters of the *market consistent* fit
+ A function **realWorldVasicekParameters** that returns the set of *real-world* parameters, given a set of *market consistent* ones.
+ A function **vasicekContCompSpot**, that takes a set of Vasicek parameters, and will derive the *terminalMaturity* year-ahead, *maturity*-year spot rate on a continously compounded basis.
+ We can call this function on the **realWorldVasicekParameters**, with an "unconditional" *terminalMaturity* (= 10,000,000 years) to return a 10-year spot rate.

This implementation is called as follows in the example project:

``` fsharp 

let real10YLongRate = 
  let calibratedVasicekParams = {
    R_2VAS_r1 = realShortPathSource_ModelData.``ShortRate.StartVal``;
    R_2VAS_r2 = realShortPathSource_ModelData.``MeanReversionLevel.StartVal``;
    R_2VAS_mudash = realShortPathSource_ModelData.``MeanReversionLevel.MuDash``;
    R_2VAS_a1 = realShortPathSource_ModelData.``ShortRate.Alpha``;
    R_2VAS_a2 = realShortPathSource_ModelData.``MeanReversionLevel.Alpha``;
    R_2VAS_s1 = realShortPathSource_ModelData.``ShortRate.Sigma``;
    R_2VAS_s2 = realShortPathSource_ModelData.``MeanReversionLevel.Sigma``;
    R_2VAS_mu = assumptions_LongRates_ModelData.``FEA_Exp_RealRate``;
    gamma = 0.0
  }
  let rwVasicekParams = realWorldVasicekParameters calibratedVasicekParams
  vasicekContCompSpot rwVasicekParams 10000000.0 10.0
  
```

So, this *real10YLongRate* is our unconditional target for the **Real Long Term Path**.

## Logistic Function Weights

The weights used when setting the target path $ k_t = \frac{1}{1+\exp^{-Bt+A}}-\frac{1}{1+\exp^A} $ can always be expressed in terms of the parameters $ (A,B) $.

This will be the sole use of the *Settings* anchored model, to pass in the values for these.  

From the example project, this is set as follows:

``` fsharp
let commonWeightsParamteters = 
  {A = settings_ModelData.``logisticWeight.Param.A``; B = settings_ModelData.``logisticWeight.Param.B`` }
```

## Outputs and Model Pushes

Now that we have defined all of the inputs to each of the four target path functions, we can run each of these and return the **Outputs** for each.  

So at the end of the "Tool Run" we will have four of these **Outputs** avaliable. We can assume that we will label these as per the intermediate method outputs in the TMT diagram, so we will have the following named outputs avaliable.  

+ NominalPath_1Y
+ NominalPath_10Y
+ InflationPath_1Y
+ RealPath_10Y

We will now define how we want to push this data to the *Anchored Models* that we will define.

### Output Parameter Types

In the example project, recall that the **Outputs** has a type signature as follows.  

``` fsharp

  // Return the final output as a record type. We will include all the evaluated intermediate parameters within this
  type TargetPathToolOutputs = 
    {
      targetPath:(float * float)[]; 
      weights:(float * float)[]; 
      meanRevRate: float;
      shortPath:(float * float)[]; 
      mediumPath:(float * float)[]; 
      longRate:float
    }  
    
```

So there are only two types of parameters we need to deal with - single numbers, and arrays of pairs of numbers (n.b. ``float`` is an alias for the .NET "Double" type).  We can map these directly to the existing types that we use for Model Pushes.  

+ ``float`` can be pushed as a **Single** value
+ ``(float * float)[]`` can be pushed as a **Vector** value - labelling the tuple as *Term* and *Value* as per past vectior pushes

### Output Anchored Models

As we have defined the downstream tools for this data long before the path setting tool was concieved, we need to fit the outputs of this tool run into a shape that will be acceptable for the existing tools. So here we will replicate the existing pushes to the models used in practice - so our four **Outputs** will need to be mapped to the three **Anchored Models** we will set up for pushing.  

These **Anchored Models**, as per the TMT diagram shall be named:

+ Output_Nominal
+ Output_Inflation
+ Output_RealLong

The outputs will be mapped to these models as per the TMT diagram, or as shown in the table below:

In [19]:
df = pd.read_json("data/PathSettingAnchors.json")
df.set_index(["Required"], inplace=True)
df[['Nominal Short Term','Nominal Long Term','Real Short Term','Real Long Term']]

,Nominal Short Term,Nominal Long Term,Real Short Term,Real Long Term
Required,,,,
Anchored Model,Output_Nominal,Output_Nominal,Output_Inflation,Output_RealLong
Named Output to push,NominalPath_1Y,NominalPath_10Y,InflationPath_1Y,RealPath_10Y


### Output Parameter Mapping

We will need to map the names of the parameters in the **Output** result using the names that we have used previously in the model pushes, where this parameter was present in the past. This can be shown in full in the table below.

In [21]:
df = pd.read_json("data/PathSettingMapping.json")
df.set_index(["Parameter"], inplace=True)
df[['NominalPath_1Y','NominalPath_1Y','InflationPath_1Y','RealPath_10Y']]

,NominalPath_1Y,NominalPath_1Y,InflationPath_1Y,RealPath_10Y
Parameter,,,,
targetPath,UserShortRatePath + Paths.ShortRate.Model,UserShortRatePath + Paths.ShortRate.Model,Paths.Inflation.InflationBreakEven.Target,Paths.Real.LongRate.Target
weights,Nominal.ShortTerm.LogisticWeights,Nominal.ShortTerm.LogisticWeights,Real.Inflation.LogisticWeights,Real.LongTerm.LogisticWeights
meanRevRate,Nominal.ShortTerm.MeanRevRate,Nominal.ShortTerm.MeanRevRate,Real.Inflation.MeanRevRate,Real.LongTerm.MeanRevRate
shortPath,Curve.YC.Fwd.ContComp,Curve.YC.Fwd.ContComp,Paths.Inflation.InflationBreakEven.Initial,Real.LongTerm.ShortPath
mediumPath,Nominal.ShortTerm.MediumPath,Nominal.ShortTerm.MediumPath,Real.Inflation.MediumPath,Real.LongTerm.MediumPath
longRate,Economy.Nominal.Shortrate.ULT.Target,Economy.Nominal.Shortrate.ULT.Target,Economy.Real.Inflation.ULT.Target,Economy.Real.10Y.ULT.Target


Note that the nominal 1Y *targetPath* is pushed twice here, under two different parameter names.